In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-05"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10669,2024-04-05,Eua Nba,20:00,Indiana Pacers,Oklahoma City Thunder,1.43,2.55,234.5,1.82,1.99,-4.5,1.88,3.34,hvZMdIJc,0.699301,0.392157,0.549451,0.502513,0.091458,0.2,0.4,NaN,NaN,219.918,62.066450,0.282225,198.586,41.018192,0.206551,197.784,207.458,189.66,242.95,0.0,0.0,0.0,0.0,0.397970,0.063101,0.395546,0.01,0.002,215.000000,0.580590,0.7,0.119410,0.57,0.114,13.596491,0.587130,0.5,-0.087130
10670,2024-04-05,Eua Nba,21:00,Chicago Bulls,New York Knicks,1.93,1.93,210.5,1.90,1.97,-2.5,2.04,2.10,rTRZgG3G,0.518135,0.518135,0.526316,0.507614,0.036269,0.0,0.2,NaN,NaN,206.994,60.079380,0.290247,204.222,32.230138,0.157819,210.326,210.386,174.73,199.92,0.0,0.0,0.0,0.0,0.000000,0.025580,0.020496,-2.73,-0.546,-1.703297,0.506201,0.4,-0.106201,1.42,0.284,3.274648,0.566816,0.8,0.233184
10671,2024-04-05,Eua Nba,21:00,Houston Rockets,Miami Heat,1.93,1.93,213.5,1.83,1.97,-2.5,2.05,2.07,MJQwgzJM,0.518135,0.518135,0.546448,0.507614,0.036269,0.4,0.4,NaN,NaN,187.600,28.677896,0.152867,279.888,80.983076,0.289341,249.092,290.362,219.35,249.31,0.0,0.0,0.0,0.0,0.000000,0.052103,0.006865,1.70,0.340,2.735294,0.506168,0.5,-0.006168,5.38,1.076,0.864312,0.437435,0.7,0.262565
10672,2024-04-05,Eua Nba,21:30,Dallas Mavericks,Golden State Warriors,1.48,2.77,227.5,1.81,2.00,2.5,1.90,2.33,tbz1d5Vi,0.675676,0.361011,0.552486,0.500000,0.036687,0.0,0.4,NaN,NaN,232.524,28.346655,0.121909,214.698,40.155543,0.187033,175.266,246.500,253.59,263.61,0.0,0.0,0.0,0.0,0.429255,0.070525,0.143762,2.70,0.540,0.888889,0.598995,0.8,0.201005,3.22,0.644,2.748447,0.494388,0.8,0.305612
10673,2024-04-05,Eua Nba,23:00,Phoenix Suns,Minnesota Timberwolves,1.66,2.04,219.5,1.86,1.95,-3.5,1.97,2.65,YBPHohBd,0.602410,0.490196,0.537634,0.512821,0.092606,0.4,0.6,NaN,NaN,223.572,45.820552,0.204948,203.504,30.371208,0.149241,179.812,206.034,157.44,200.72,0.0,0.0,0.0,0.0,0.145244,0.033407,0.208153,0.32,0.064,10.312500,0.691911,0.9,0.208089,1.90,0.380,2.736842,0.615808,0.7,0.084192
10674,2024-04-05,Europa Euroliga,15:30,Real Madrid,Baskonia,1.32,3.57,171.5,1.91,1.91,-7.5,1.91,7.00,W8pSfmzC,0.757576,0.280112,0.523560,0.523560,0.037688,0.8,1.0,NaN,NaN,124.616,26.476078,0.212461,178.452,93.949392,0.526469,111.904,245.456,115.24,167.20,0.0,0.0,0.0,0.0,0.650712,0.000000,0.807895,-0.32,-0.064,-5.000000,0.847491,0.9,0.052509,-2.36,-0.472,-5.444915,0.396213,0.4,0.003787
10675,2024-04-05,Alemanha Bbl,15:00,Tubingen,Rostock,1.90,1.90,179.5,1.86,1.90,-1.5,2.02,2.03,KKgijAMo,0.526316,0.526316,0.537634,0.526316,0.052632,0.6,0.6,NaN,NaN,352.098,318.090747,0.903415,182.044,34.861083,0.191498,218.252,217.624,156.64,180.60,0.0,0.0,0.0,0.0,0.000000,0.015045,0.003492,-3.22,-0.644,-1.397516,0.380217,0.3,-0.080217,-5.00,-1.000,-0.900000,0.000000,0.0,0.000000
10676,2024-04-05,Austrália Nbl1 Sul,06:00,Nunawading,Ringwood,2.53,1.48,176.5,1.83,1.87,-3.5,2.00,1.72,fyDxCYAN,0.395257,0.675676,0.546448,0.534759,0.070933,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.370305,0.015289,0.106446,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10677,2024-04-05,Austrália Nbl1 Oeste,09:30,Kalamunda Eastern Suns,East Perth Eagles,1.59,2.24,179.5,1.89,1.89,1.5,1.92,2.13,6JUWIlaQ,0.628931,0.446429,0.529101,0.529101,0.075359,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.240010,0.000000,0.073330,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10678,202

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,20:00,Eua Nba,Indiana Pacers,Oklahoma City Thunder,234.5,1.82,1.0000,Over
1,21:00,Eua Nba,Chicago Bulls,New York Knicks,210.5,1.90,1.0000,Over
2,21:00,Eua Nba,Houston Rockets,Miami Heat,213.5,1.83,1.0000,Over
3,21:30,Eua Nba,Dallas Mavericks,Golden State Warriors,227.5,1.81,1.0000,Over
4,23:00,Eua Nba,Phoenix Suns,Minnesota Timberwolves,219.5,1.86,1.0000,Over
5,15:30,Europa Euroliga,Real Madrid,Baskonia,171.5,1.91,1.0000,Over
6,15:00,Alemanha Bbl,Tubingen,Rostock,179.5,1.86,1.0000,Over
7,09:30,Austrália Nbl1 Oeste,Kalamunda Eastern Suns,East Perth Eagles,179.5,1.89,1.0000,Over
8,15:00,Europa Liga Bnxt,Liege,Den Bosch,158.5,1.84,0.9989,Over
9,15:00,França Pro B,Angers,Denain-Voltaire,160.5,1.85,1.0000,Over
